In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# today can reproduce GPT-2 in about an hour
# GPT-2 paper is pretty light on details, so we reference the GPT-3 paper as well (not huge change in architecture)

* GPT-2 - [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
* GPT-3 - [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165)

In [ ]:
# original written in tensorflow
# karpathy not a fan